In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyxdf
import joblib

In [2]:
no_of_files=12

all_xdfs = [pyxdf.load_xdf('dataset/A'+str(f)+'.xdf')[0] for f in range(1,no_of_files+1)]

In [3]:
def xdf_extract(i,eventWindow,channel_length,eeg_channel,stim_channel):
    dataset_arr=np.empty((0,eventWindow,channel_length))
    start_time=float(i[eeg_channel]['footer']['info']['first_timestamp'][0])
    end_time=float(i[eeg_channel]['footer']['info']['last_timestamp'][0])
    no_samples=float(i[eeg_channel]['footer']['info']['sample_count'][0])
    Fs=i[eeg_channel]['info']['effective_srate']
    try:
        Ts=1/Fs
    except ZeroDivisionError:
        print('ZeroDivisionError')
        return
    time_stamps=(i[eeg_channel]['time_stamps']-start_time)/(end_time-start_time)*no_samples*Ts
    time_index=(i[eeg_channel]['time_stamps']-start_time)/(end_time-start_time)*no_samples
    first_marker_time=(float(i[stim_channel]['footer']['info']['first_timestamp'][0])-start_time)/(end_time-start_time)*no_samples*Ts
    last_marker_time=(float(i[stim_channel]['footer']['info']['last_timestamp'][0])-start_time)/(end_time-start_time)*no_samples*Ts
    markers=list()
    for j in i[stim_channel]['time_series']:
        markers.append(j[0])
    markers_time=list((i[stim_channel]['time_stamps']-start_time)/(end_time-start_time)*no_samples*Ts)
    markers_index=list((i[stim_channel]['time_stamps']-start_time)/(end_time-start_time)*no_samples)

    for e in range(1,len(markers)-1):
        try:
            data=(i[eeg_channel]['time_series'][(np.where(time_stamps>=markers_time[e])),:])[:,0:eventWindow,:]#[0:3,:]#[:,0:eventWindow,:].reshape(eventWindow,8).reshape(1,eventWindow,8)
            # print(data.shape)
            dataset_arr=np.vstack((dataset_arr,data))
        except ValueError as v:
            print(data.shape,'ValueError',v)
            return
    return dataset_arr,markers[1:-1]

In [9]:
eeg_data = None
eeg_markers = None
for i,xdf in enumerate(all_xdfs[1:]):
    if xdf != None:
        if i == 0:
            eeg = xdf_extract(xdf, 250, 8, 0, 1)
            eeg_data = eeg[0]
            eeg_markers = eeg[1]
        else:
            eeg = xdf_extract(xdf, 250, 8, 0, 1)
            eeg_data = np.vstack((eeg_data,eeg[0]))
            eeg_markers = eeg_markers + eeg[1]
eeg_data = np.transpose(eeg_data, [0,2,1])
print(eeg_data.shape, len(eeg_markers))

(3945, 8, 250) 3945


In [11]:
eeg_data.shape

(3945, 8, 250)

In [12]:
eeg = {'X':eeg_data, 'y':eeg_markers}

# eeg_file = open('altered_data/merged_eeg.sav','w')

joblib.dump(eeg,'altered_data/merged_eeg.sav')

['altered_data/merged_eeg.sav']